In [14]:
from pathlib import Path
import pandas as pd

In [15]:
df = pd.read_csv("adult_test.csv", index_col=None)

In [16]:
df.head()

,Unnamed: 0,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income-per-year
0,14214,32,Private,167106,HS-grad,9,Married-civ-spouse,Sales,Husband,Asian-Pac-Islander,Male,0,0,40,Hong,<=50K
1,23012,64,Self-emp-inc,307786,1st-4th,2,Married-civ-spouse,Sales,Husband,White,Male,0,0,20,United-States,<=50K
2,4984,27,Private,194652,HS-grad,9,Never-married,Craft-repair,Own-child,Black,Male,0,0,40,United-States,<=50K
3,30559,27,Private,141545,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,1902,45,United-States,<=50K
4,10130,27,Private,146460,Some-college,10,Never-married,Machine-op-inspct,Not-in-family,White,Female,0,0,40,United-States,<=50K


In [17]:
print(df.dtypes)

Unnamed: 0          int64
age                 int64
workclass          object
fnlwgt              int64
education          object
education-num       int64
marital-status     object
occupation         object
relationship       object
race               object
sex                object
capital-gain        int64
capital-loss        int64
hours-per-week      int64
native-country     object
income-per-year    object
dtype: object


In [18]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9769 entries, 0 to 9768
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       9769 non-null   int64 
 1   age              9769 non-null   int64 
 2   workclass        9192 non-null   object
 3   fnlwgt           9769 non-null   int64 
 4   education        9769 non-null   object
 5   education-num    9769 non-null   int64 
 6   marital-status   9769 non-null   object
 7   occupation       9189 non-null   object
 8   relationship     9769 non-null   object
 9   race             9769 non-null   object
 10  sex              9769 non-null   object
 11  capital-gain     9769 non-null   int64 
 12  capital-loss     9769 non-null   int64 
 13  hours-per-week   9769 non-null   int64 
 14  native-country   9587 non-null   object
 15  income-per-year  9769 non-null   object
dtypes: int64(7), object(9)
memory usage: 1.2+ MB
None


## Create statements from source CSV file

In [3]:
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd

In [29]:
def _create_table_code(data_frame, table_name, db_path, add_mlwhatif_serial, index_col):
    
    # column names of dataframe
    col_names = [x for x in data_frame.columns.values if x != "index-mlwhatif"]

    if "index-mlwhatif" in data_frame.columns.values:
        col_names.append("index-mlwhatif")

    # DROP statement
    drop_old_table = f"DROP TABLE IF EXISTS {table_name};"
    
    # set the duckdb engine
    engine_str = "duckdb:///" + db_path
    duckdb_engine = create_engine(engine_str)
    
    # CREATE statements
    create_statements = []
    
    # get the schema for the duckdb database
    schema = pd.io.sql.get_schema(data_frame, name=table_name, con=duckdb_engine)
    
    # add serial id column
    new_column_id1 = "CREATE SEQUENCE index-mlwhatif-seq START 1;"
    new_column_id2 = "\"index-mlwhatif\" INTEGER PRIMARY KEY DEFAULT NEXTVAL('index-mlwhatif-seq')"
    
    # alter index column name to 'index-mlwhatif'
    alter_column_idx = f"ALTER TABLE {table_name} RENAME COLUMN '{index_col}' TO 'index-mlwhatif';"

    statement = schema.rstrip()[:-1]

    new_create_statement = statement.rstrip() + ',\n\t' + new_column_id2 + '\n)'
    
    if add_mlwhatif_serial and index_col == -1:
        create_statements = [new_column_id1, new_create_statement]
    elif add_mlwhatif_serial and index_col != -1:
        create_statements = [schema, alter_column_idx]
    else: 
        create_statements = [schema]
        
    return col_names, drop_old_table, create_statements



In [30]:
data_frame = pd.read_csv('adult_test.csv', index_col=None)
db_path = 'my_duckdb'

In [31]:
# add_mlwhatif_serial and index_col == -1
col_names, drop_old_table, create_statements = _create_table_code(data_frame, 'test', db_path, True, -1)
print(f"Columns of the dataframe are: {col_names}\n")
print(f"Drop table statement: {drop_old_table}\n")
print("Create table statements:")
for statement in create_statements:
    print(statement)

Columns of the dataframe are: ['Unnamed: 0', 'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income-per-year']

Drop table statement: DROP TABLE IF EXISTS test;

Create table statements:
CREATE SEQUENCE index-mlwhatif-seq START 1;

CREATE TABLE test (
	"Unnamed: 0" BIGINT, 
	age BIGINT, 
	workclass TEXT, 
	fnlwgt BIGINT, 
	education TEXT, 
	"education-num" BIGINT, 
	"marital-status" TEXT, 
	occupation TEXT, 
	relationship TEXT, 
	race TEXT, 
	sex TEXT, 
	"capital-gain" BIGINT, 
	"capital-loss" BIGINT, 
	"hours-per-week" BIGINT, 
	"native-country" TEXT, 
	"income-per-year" TEXT,
	"index-mlwhatif" INTEGER PRIMARY KEY DEFAULT NEXTVAL('index-mlwhatif-seq')
)


In [32]:
# add_mlwhatif_serial and index_col != -1
col_names, drop_old_table, create_statements = _create_table_code(data_frame, 'test', db_path, True, 'Unnamed: 0')
print(f"Columns of the dataframe are: {col_names}\n")
print(f"Drop table statement: {drop_old_table}\n")
print("Create table statements:")
for statement in create_statements:
    print(statement)

Columns of the dataframe are: ['Unnamed: 0', 'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income-per-year']

Drop table statement: DROP TABLE IF EXISTS test;

Create table statements:

CREATE TABLE test (
	"Unnamed: 0" BIGINT, 
	age BIGINT, 
	workclass TEXT, 
	fnlwgt BIGINT, 
	education TEXT, 
	"education-num" BIGINT, 
	"marital-status" TEXT, 
	occupation TEXT, 
	relationship TEXT, 
	race TEXT, 
	sex TEXT, 
	"capital-gain" BIGINT, 
	"capital-loss" BIGINT, 
	"hours-per-week" BIGINT, 
	"native-country" TEXT, 
	"income-per-year" TEXT
)


ALTER TABLE test RENAME COLUMN 'Unnamed: 0' TO 'index-mlwhatif';


In [37]:
# add_mlwhatif_serial and index_col != -1
col_names, drop_old_table, create_statements = _create_table_code(data_frame, 'test', db_path, False, 'Unnamed: 0')
print(f"Columns of the dataframe are: {col_names}\n")
print(f"Drop table statement: {drop_old_table}\n")
print("Create table statements:")
for statement in create_statements:
    print(statement)

Columns of the dataframe are: ['Unnamed: 0', 'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income-per-year']

Drop table statement: DROP TABLE IF EXISTS test;

Create table statements:

CREATE TABLE test (
	"Unnamed: 0" BIGINT, 
	age BIGINT, 
	workclass TEXT, 
	fnlwgt BIGINT, 
	education TEXT, 
	"education-num" BIGINT, 
	"marital-status" TEXT, 
	occupation TEXT, 
	relationship TEXT, 
	race TEXT, 
	sex TEXT, 
	"capital-gain" BIGINT, 
	"capital-loss" BIGINT, 
	"hours-per-week" BIGINT, 
	"native-country" TEXT, 
	"income-per-year" TEXT
)




In [64]:
def get_sql_code_csv(path_to_csv, table_name, null_symbols=None, delimiter=",", header=True,
                         drop_old=False, add_mlwhatif_serial=False, index_col=-1):

    if null_symbols is None:
        null_symbols = ["?"]

    data_frame = pd.read_csv(path_to_csv, index_col=None, header=0, encoding='utf-8')

    names, drop_old_table, create_table = _create_table_code(data_frame, table_name, db_path,
                                                            add_mlwhatif_serial,
                                                            index_col=index_col)

    if len(null_symbols) != 1:
        raise NotImplementedError("Currently only ONE null symbol supported!")

    add_data = f"COPY {table_name} " \
                f"FROM '{path_to_csv}' (" \
                f"DELIMITER '{delimiter}', NULL '{null_symbols[0]}', FORMAT CSV, HEADER {'TRUE' if header else 'FALSE'});"

    if drop_old:
        return [drop_old_table] + create_table + [add_data]
    
    return names, create_table + [add_data]

In [65]:
path_to_csv = 'adult_test.csv'
table_name = 'test'

In [66]:
names, statements = get_sql_code_csv(path_to_csv, table_name)
print(names)
for statement in statements: 
    print(statement)

['Unnamed: 0', 'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income-per-year']

CREATE TABLE test (
	"Unnamed: 0" BIGINT, 
	age BIGINT, 
	workclass TEXT, 
	fnlwgt BIGINT, 
	education TEXT, 
	"education-num" BIGINT, 
	"marital-status" TEXT, 
	occupation TEXT, 
	relationship TEXT, 
	race TEXT, 
	sex TEXT, 
	"capital-gain" BIGINT, 
	"capital-loss" BIGINT, 
	"hours-per-week" BIGINT, 
	"native-country" TEXT, 
	"income-per-year" TEXT
)


COPY test FROM 'adult_test.csv' (DELIMITER ',', NULL '?', FORMAT CSV, HEADER TRUE);


## Transform

In [ ]:
def get_unique_id(self):
        self.id += 1
        return self.id - 1

In [ ]:
def simple_imputer(table, column_name, mode):
    table_name = f"block_impute_fit_{get_unique_id()}_most_frequent"
    if mode == 'max':
        sql_code = f"WITH counts_help AS (\n" \
                   f"\tSELECT {column_name}, COUNT(*) AS count\n" \
                   f"\tFROM {table} \n" \
                   f"\tGROUP BY {column_name}\n" \
                   f")\n" \
                   f"SELECT {column_name} AS most_frequent \n" \
                   f"FROM counts_help\n" \
                   f"WHERE counts_help.count = (SELECT MAX(count) FROM counts_help)\n" \
                   f"LIMIT 1"
    elif mode == 'mean':
        table_name = f"block_impute_fit_{self.get_unique_id()}_mean"
        sql_code = f"SELECT (SELECT AVG({column_name}) FROM {table}) AS {column_name}"
    elif mode == 'one_hot_encoding':
        table_name = f"block_one_hot_fit_{self.get_unique_id()}"
        sql_code = f"SELECT {col}, \n" \
                   f"(array_fill(0, ARRAY[\"rank\" - 1]) || 1 ) || " \
                   f"array_fill(0, ARRAY[ CAST((select COUNT(distinct({col})) FROM {table}) AS int) - " \
                   f"(\"rank\")]) AS {col[:-1]}_one_hot\" \n" \
                   f"\tFROM (\n" \
                   f"\tSELECT {col}, CAST(ROW_NUMBER() OVER() AS int) AS \"rank\" \n" \
                   f"\tFROM (SELECT distinct({col}) FROM {table}) oh\n" \
                   f") one_hot_help"   
        
    block_name, sql_code = self.wrap_in_sql_obj(sql_code, block_name=table_name)
    return self.materialize_if_possible(block_name, sql_code)